# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [149]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json 
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [150]:
#read csv from previous activity
output_file = pd.read_csv("../output_data/weather_df.csv")
output_file.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,0,emba,99,KZ,1584643002,71,48.83,58.14,36.41,7.65
1,1,mataura,50,NZ,1584643003,68,-46.19,168.86,50.00,14.00
2,2,chuy,3,UY,1584643003,50,-33.70,-53.46,77.09,16.84
3,3,esperance,75,AU,1584643003,93,-33.87,121.90,59.00,4.70
4,4,kurilsk,79,RU,1584643004,99,45.23,147.88,36.05,7.31


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [151]:
gmaps.configure(api_key=g_key)

In [152]:
#help(gmaps.heatmap_layer)

In [153]:
locations=output_file[["Lat", "Long"]]
weight = output_file["Humidity"]
figure = gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weight, 
                            dissipating=False,
                            max_intensity =5,
                            point_radius =1)
figure.add_layer(heat_layer)

In [154]:
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [155]:
#narrow down data to: 
#max temperature <= 80 degrees but > 70
#wind speeds <10
#zero cloudiness
#drop na's
ideal_temps = (output_file["Max Temp"]<= 80) &( output_file["Max Temp"]> 70)
ideal_wind_spd = (output_file["Wind Speed"] < 10)
ideal_cloud_cov = (output_file["Cloudiness"] < 1)
ideal_weather = ideal_temps & ideal_wind_spd & ideal_cloud_cov
ideal_weather_df = output_file[ideal_weather]
ideal_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
282,282,adrar,0,MR,1584643071,15,20.50,-10.07,79.43,9.31
291,291,khudaganj,0,IN,1584643073,41,28.15,79.72,70.25,5.23
399,399,richards bay,0,ZA,1584643098,75,-28.78,32.04,77.81,5.59
405,405,yulara,0,AU,1584643100,27,-25.24,130.99,77.00,6.93
474,474,marawi,0,PH,1584643116,85,8.00,124.29,78.01,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
*Plot markers on top of the heatmap.

In [156]:
hotel_df=ideal_weather_df
hotel_df

#create params dictionary
params ={
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    #get lats and longs
    lat=row["Lat"]
    long=row["Long"]
    params["location"]=f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    

Retrieving Results for Index 282: adrar.
Missing field/result... skipping.
------------
Retrieving Results for Index 291: khudaganj.
Closest hotel is Virat Bhawan.


/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


------------
Retrieving Results for Index 399: richards bay.
Closest hotel is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Index 405: yulara.
Closest hotel is Desert Gardens Hotel - Ayers Rock Resort.
------------
Retrieving Results for Index 474: marawi.
Closest hotel is Derogongan Residence.
------------


In [157]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed,Hotel Name
282,282,adrar,0,MR,1584643071,15,20.50,-10.07,79.43,9.31,NaN
291,291,khudaganj,0,IN,1584643073,41,28.15,79.72,70.25,5.23,Virat Bhawan
399,399,richards bay,0,ZA,1584643098,75,-28.78,32.04,77.81,5.59,BON Hotel Waterfront Richards Bay
405,405,yulara,0,AU,1584643100,27,-25.24,130.99,77.00,6.93,Desert Gardens Hotel - Ayers Rock Resort
474,474,marawi,0,PH,1584643116,85,8.00,124.29,78.01,1.01,Derogongan Residence


In [158]:
# NOTE: Do not change any of the code in this cell
locations=hotel_df[["Lat", "Long"]]
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [159]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)

# Display Map
figure.add_layer(markers)
figure.add_layer(hotel_layer)
figure

Figure(layout=FigureLayout(height='420px'))